In [1]:
(in-package "ACL2")

 "ACL2"


```bash
cert.pl $ACL2_SYSTEM_BOOKS/kestrel/fty/top
```

In [2]:
(include-book "kestrel/fty/top" :DIR :SYSTEM)


Summary
Form:  ( INCLUDE-BOOK "kestrel/fty/top" ...)
Rules: NIL
Time:  1.30 seconds (prove: 0.00, print: 0.00, other: 1.29)
 "/home/acl2/books/kestrel/fty/top.lisp"


In [3]:
; Action type (sum/discriminated union)
(fty::deftagsum tool-action
  (:search ((query stringp)
            (estimated-cost rationalp)))
  (:llm ((prompt stringp)
         (token-count natp)
         (estimated-cost rationalp)))
  (:calculator ((expression any-p)
                (estimated-cost rationalp)))
  (:final-answer ((answer stringp))))

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [5]:
; Observation record - what happened after an action
(fty::defprod observation
  ((action-name symbolp)
   (action-input alistp)
   (estimated-cost rationalp)
   (actual-result any-p)
   (actual-cost rationalp)
   (success booleanp)
   (error-msg stringp)))



HARD ACL2 ERROR in FTY::GET-PRED/FIX/EQUIV:  Type ALISTP doesn't have
an associated fixing function.  Please provide that association using
FTY::DEFFIXTYPE.




ACL2 Error [Evaluation] in ( MAKE-EVENT (FTY::DEFPROD-FN ...)):  Evaluation
aborted.  To debug see :DOC print-gv, see :DOC trace, and see :DOC
wet.


Summary
Form:  ( MAKE-EVENT (FTY::DEFPROD-FN ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( MAKE-EVENT (FTY::DEFPROD-FN ...)):  See :DOC
failure.

******** FAILED ********


In [ ]:
; List of observations
(fty::deflist observation-list
  :elt-type observationp
  :true-listp t)

In [ ]:
; Quota tracking for each tool type
; Associates tool name to remaining call count
(fty::defalist tool-quota
  :key-type symbolp
  :val-type natp
  :true-listp t)

In [ ]:
; Agent state - complete execution context
(fty::defprod agent-st
  ((goal stringp
          "The task the agent is solving")
   (reasoning string-listp
              "Trace of reasoning steps")
   (observations observation-list-p
                 "Complete record of all actions taken")
   (budget-remaining rationalp
                     "Unspent budget in dollars")
   (quota-remaining tool-quota-p
                    "Call count remaining per tool")
   (step-count natp
               "Number of iterations completed")
   (max-steps natp
              "Upper bound on iterations (termination guarantee)")
   (final-answer (or stringp null)
                 "Answer, or nil if not yet found")))

In [ ]:
(defun budget-remaining (st)
  "Read the remaining budget from agent state"
  (declare (xargs :guard (agent-st-p st)))
  (agent-st->budget-remaining st))

In [ ]:
(defun budget-sufficient-for-cost (cost st)
  "Check: is there enough budget to pay this cost?"
  (declare (xargs :guard (and (rationalp cost) (agent-st-p st))))
  (<= cost (budget-remaining st)))

In [ ]:
(defun deduct-from-budget (cost st)
  "Spend cost from budget. Precondition: cost <= budget-remaining"
  (declare (xargs :guard (and (rationalp cost)
                              (agent-st-p st)
                              (budget-sufficient-for-cost cost st))))
  (change-agent-st st
                   :budget-remaining (- (budget-remaining st) cost)))

In [ ]:
(defun quota-for-tool (tool-name quota)
  "Look up remaining calls allowed for this tool"
  (declare (xargs :guard (and (symbolp tool-name) (tool-quota-p quota))))
  (let ((entry (assoc tool-name quota)))
    (if entry (cdr entry) 0)))

In [ ]:
(defun quota-available-for-tool (tool-name st)
  "Check: are there calls remaining for this tool?"
  (declare (xargs :guard (and (symbolp tool-name) (agent-st-p st))))
  (> (quota-for-tool tool-name (agent-st->quota-remaining st)) 0))

In [ ]:
(defun decrement-quota-for-tool (tool-name st)
  "Use one call from this tool's quota. Precondition: quota-available-for-tool"
  (declare (xargs :guard (and (symbolp tool-name)
                              (agent-st-p st)
                              (quota-available-for-tool tool-name st))))
  (let* ((old-quota (agent-st->quota-remaining st))
         (current (quota-for-tool tool-name old-quota))
         (new-quota (cons (cons tool-name (- current 1))
                          (remove-assoc tool-name old-quota))))
    (change-agent-st st :quota-remaining new-quota)))

In [ ]:
; SEARCH TOOL
(defun search-tool-cost (query-string)
  "Search costs $5 per query, independent of query length"
  (declare (xargs :guard (stringp query-string)))
  5)

In [ ]:
(defun search-tool-can-execute (st)
  "Search can execute if: budget covers cost AND quota available"
  (declare (xargs :guard (agent-st-p st)))
  (and (budget-sufficient-for-cost (search-tool-cost "dummy") st)
       (quota-available-for-tool 'search st)))

In [ ]:
(defun search-tool-result-valid (result)
  "Search result is valid: non-empty string"
  (declare (xargs :guard t))
  (and (stringp result) (> (length result) 0)))

In [ ]:
; LLM TOOL
(defun llm-tool-cost (token-count)
  "LLM costs $0.00001 per token"
  (declare (xargs :guard (natp token-count)))
  (* token-count 1/100000))  ; Using rationals for exact arithmetic

In [ ]:
(defun llm-tool-can-execute (token-count st)
  "LLM can execute if: budget covers tokens AND quota available"
  (declare (xargs :guard (and (natp token-count) (agent-st-p st))))
  (and (budget-sufficient-for-cost (llm-tool-cost token-count) st)
       (quota-available-for-tool 'llm st)))

In [ ]:
(defun llm-tool-result-valid (result)
  "LLM result is valid: non-empty string"
  (declare (xargs :guard t))
  (and (stringp result) (> (length result) 0)))

In [ ]:
; CALCULATOR TOOL
(defun calculator-tool-cost (expression)
  "Calculator is free"
  (declare (xargs :guard t))
  0)

In [ ]:
(defun calculator-tool-can-execute (st)
  "Calculator always executable (no cost, no quota limit)"
  (declare (xargs :guard (agent-st-p st)))
  t)

In [ ]:
(defun calculator-tool-result-valid (result)
  "Calculator result is valid: any rational number"
  (declare (xargs :guard t))
  (rationalp result))

In [ ]:
(defun record-action-taken (tool-name input estimated-cost actual-result actual-cost success error-msg st)
  "Record that an action was attempted, update state"
  (declare (xargs :guard (and (symbolp tool-name)
                              (alistp input)
                              (rationalp estimated-cost)
                              (rationalp actual-cost)
                              (booleanp success)
                              (or (stringp error-msg) (null error-msg))
                              (agent-st-p st))))
  
  (let* ((obs (make-observation 
               :action-name tool-name
               :action-input input
               :estimated-cost estimated-cost
               :actual-result actual-result
               :actual-cost actual-cost
               :success success
               :error-msg error-msg))
         
         (observations-with-new (cons obs (agent-st->observations st)))
         
         (st-after-cost (deduct-from-budget actual-cost st))
         
         (st-after-quota (decrement-quota-for-tool tool-name st-after-cost))
         
         (st-after-step (change-agent-st st-after-quota
                                         :step-count (+ 1 (agent-st->step-count st-after-quota))
                                         :observations observations-with-new)))
    
    st-after-step))

In [ ]:
; Theorem 1: Deducting sufficient cost keeps budget non-negative
(defthm deduct-preserves-non-negative-budget
  (implies (and (rationalp cost)
                (rationalp budget)
                (<= cost budget))
           (>= (- budget cost) 0))
  :rule-classes :linear)

In [ ]:
; Theorem 2: After deduct-from-budget, budget is smaller
(defthm deduct-decreases-budget
  (implies (and (agent-st-p st)
                (rationalp cost)
                (> cost 0)
                (budget-sufficient-for-cost cost st))
           (< (agent-st->budget-remaining (deduct-from-budget cost st))
              (agent-st->budget-remaining st)))
  :rule-classes :linear)

In [ ]:
; Theorem 3: Budget never goes negative
(defthm record-action-preserves-budget-non-negative
  (implies (and (agent-st-p st)
                (rationalp actual-cost)
                (budget-sufficient-for-cost actual-cost st))
           (>= (agent-st->budget-remaining 
                (record-action-taken 'search nil actual-cost 'result actual-cost t nil st))
               0))
  :rule-classes :linear)

In [ ]:
; Theorem 4: Quota for tool after decrement is strictly less
(defthm decrement-quota-reduces-tool-count
  (implies (and (symbolp tool-name)
                (agent-st-p st)
                (quota-available-for-tool tool-name st))
           (< (quota-for-tool tool-name 
                              (agent-st->quota-remaining 
                               (decrement-quota-for-tool tool-name st)))
              (quota-for-tool tool-name (agent-st->quota-remaining st))))
  :rule-classes :linear)

In [ ]:
; Theorem 5: Quota never goes negative
(defthm quota-never-negative-after-valid-decrement
  (implies (and (symbolp tool-name)
                (agent-st-p st)
                (quota-available-for-tool tool-name st))
           (>= (quota-for-tool tool-name 
                               (agent-st->quota-remaining 
                                (decrement-quota-for-tool tool-name st)))
               0))
  :rule-classes :linear)

In [ ]:
; Theorem 6: Step count strictly increases
(defthm step-count-increases
  (implies (natp step-count)
           (< step-count (+ 1 step-count)))
  :rule-classes :linear)

In [ ]:
; Theorem 7: Loop measure (max-steps - step-count) decreases
(defthm loop-measure-decreases
  (implies (and (natp step-count)
                (natp max-steps)
                (< step-count max-steps))
           (< (- max-steps (+ 1 step-count))
              (- max-steps step-count)))
  :rule-classes :linear)

In [ ]:
(defun react-loop (st llm-function)
  "Main agent execution loop.
   
   TERMINATION GUARANTEE: Loop measure is (max-steps - step-count).
   Each iteration increments step-count, so measure strictly decreases.
   ACL2 proves this loop terminates.
   
   BUDGET GUARANTEE: Only execute tool if budget-sufficient-for-cost.
   record-action-taken deducts actual cost. By theorem deduct-preserves-non-negative,
   budget never goes negative.
   
   QUOTA GUARANTEE: Only execute tool if quota-available-for-tool.
   decrement-quota-for-tool decrements quota. By theorem quota-never-negative,
   quota never goes negative.
   "
  (declare (xargs :mode :program  ; Allow external LLM calls
                  :measure (- (agent-st->max-steps st)
                              (agent-st->step-count st))))
  
  ; Termination condition: max steps reached or final answer found
  (if (or (>= (agent-st->step-count st) (agent-st->max-steps st))
          (not (null (agent-st->final-answer st))))
      st
      
      ; Step 1: LLM generates next action
      (let ((action (funcall llm-function st)))
        
        ; Step 2: Dispatch based on action type
        (case (car action)
          
          ; ============ SEARCH ACTION ============
          (:search
           (if (search-tool-can-execute st)
               (let* ((query (alist-get 'query (cdr action)))
                      (search-result "Search result stub")
                      (actual-cost (search-tool-cost query)))
                 
                 (if (search-tool-result-valid search-result)
                     (let ((st-updated (record-action-taken 'search (cdr action) 
                                                           (search-tool-cost query)
                                                           search-result
                                                           actual-cost
                                                           t nil st)))
                       (react-loop st-updated llm-function))
                     
                     ; Result validation failed - reject and retry
                     (react-loop st llm-function)))
               
               ; Precondition failed (budget/quota) - reject and retry
               (react-loop st llm-function)))
          
          ; ============ LLM ACTION ============
          (:llm
           (let ((token-count (alist-get 'token-count (cdr action))))
             (if (llm-tool-can-execute token-count st)
                 (let* ((llm-result "LLM response stub")
                        (actual-cost (llm-tool-cost token-count)))
                   
                   (if (llm-tool-result-valid llm-result)
                       (let ((st-updated (record-action-taken 'llm (cdr action)
                                                             actual-cost
                                                             llm-result
                                                             actual-cost
                                                             t nil st)))
                         (react-loop st-updated llm-function))
                       
                       (react-loop st llm-function)))
                 
                 (react-loop st llm-function))))
          
          ; ============ CALCULATOR ACTION ============
          (:calculator
           (if (calculator-tool-can-execute st)
               (let* ((expr (alist-get 'expression (cdr action)))
                      (calc-result 42)  ; Stub
                      (actual-cost 0))
                 
                 (if (calculator-tool-result-valid calc-result)
                     (let ((st-updated (record-action-taken 'calculator (cdr action)
                                                           0
                                                           calc-result
                                                           actual-cost
                                                           t nil st)))
                       (react-loop st-updated llm-function))
                     
                     (react-loop st llm-function)))
               
               (react-loop st llm-function)))
          
          ; ============ FINAL ANSWER ACTION ============
          (:final-answer
           (let ((answer (alist-get 'answer (cdr action))))
             (change-agent-st st :final-answer answer)))
          
          ; ============ UNKNOWN ACTION ============
          (otherwise st)))))

In [ ]:
(defun alist-get (key alist)
  "Get value from alist, return 0 if not found.
   DOMAIN-SPECIFIC: Only used for action-input parsing where 0 is safe default.
   NOT a general library function."
  (declare (xargs :guard (alistp alist)))
  (let ((pair (assoc key alist)))
    (if pair (cdr pair) 0)))

In [ ]:
(defun init-agent (goal initial-budget max-steps-allowed)
  "Create initial agent state with full budget and quota"
  (declare (xargs :guard (and (stringp goal)
                              (rationalp initial-budget)
                              (natp max-steps-allowed))))
  
  (make-agent-st 
    :goal goal
    :reasoning nil
    :observations nil
    :budget-remaining initial-budget
    :quota-remaining '((search . 50)      ; 50 searches allowed
                       (llm . 100)         ; 100 LLM calls allowed
                       (calculator . 1000)) ; 1000 calculator calls allowed
    :step-count 0
    :max-steps max-steps-allowed
    :final-answer nil))

In [ ]:
(defun simple-thought-generator (st)
  "Example: LLM always suggests searching for the goal"
  (declare (xargs :mode :program
                  :guard (agent-st-p st)))
  (let ((goal (agent-st->goal st)))
    (list :search 
          (list (cons 'query goal)
                (cons 'estimated-cost 5)))))

In [ ]:
(defun agent-run-example ()
  "Execute example agent, display results"
  (declare (xargs :mode :program))
  (let* ((initial-st (init-agent "What is the capital of France?" 100 10))
         (final-st (react-loop initial-st #'simple-thought-generator)))
    
    (progn
      (cw "~%══════════════════════════════════════════════════════════════~%")
      (cw "AGENT EXECUTION REPORT~%")
      (cw "══════════════════════════════════════════════════════════════~%")
      (cw "Goal: ~s~%" (agent-st->goal final-st))
      (cw "Final Answer: ~s~%" (agent-st->final-answer final-st))
      (cw "~%RESOURCE USAGE:~%")
      (cw "  Steps taken: ~d / ~d~%" 
           (agent-st->step-count final-st) (agent-st->max-steps final-st))
      (cw "  Budget spent: $~d~%" (- 100 (agent-st->budget-remaining final-st)))
      (cw "  Budget remaining: $~d~%" (agent-st->budget-remaining final-st))
      (cw "  Search calls used: ~d / 50~%" 
           (- 50 (quota-for-tool 'search (agent-st->quota-remaining final-st))))
      (cw "  LLM calls used: ~d / 100~%" 
           (- 100 (quota-for-tool 'llm (agent-st->quota-remaining final-st))))
      (cw "~%VERIFICATION STATUS:~%")
      (cw "  Budget non-negative: ~s~%" 
           (>= (agent-st->budget-remaining final-st) 0))
      (cw "  Quotas non-negative: ~s~%"
           (and (>= (quota-for-tool 'search (agent-st->quota-remaining final-st)) 0)
                (>= (quota-for-tool 'llm (agent-st->quota-remaining final-st)) 0)))
      (cw "  Loop terminated: ~s~%"
           (or (>= (agent-st->step-count final-st) (agent-st->max-steps final-st))
               (not (null (agent-st->final-answer final-st)))))
      (cw "══════════════════════════════════════════════════════════════~%~%"))))